In [9]:
# !pip install gensim

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
os.environ['NVIDIA_VISIBLE_DEVICES'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ


environ{'PATH': '/opt/conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'HOSTNAME': '78353253e378',
        'TERM': 'xterm-color',
        'NVIDIA_VISIBLE_DEVICES': '1',
        'CUDA_VERSION': '9.0.176',
        'CUDA_PKG_VERSION': '9-0=9.0.176-1',
        'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64',
        'NVIDIA_DRIVER_CAPABILITIES': 'compute,utility',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=9.0',
        'NCCL_VERSION': '2.4.2',
        'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs',
        'CUDNN_VERSION': '7.4.2.24',
        'LANG': 'C.UTF-8',
        'HOME': '/root',
        'JPY_PARENT_PID': '6',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
        'CUDA_VISIBLE_DEVICES': '1'}

In [3]:
from train import ModelLoader, load_and_save_params, MetricLoader
from datasets.cvpr2016_cub_loader import Cvpr2016CubLoader
from common.util import Namespace
import numpy as np
from tqdm import tqdm
from scipy.spatial import cKDTree, KDTree
import time
from common.metrics import ap_at_k_prototypes
import os, pwd
from common.pretrained_models import InceptionV3Loader, InceptionV2Loader
from common.pretrained_models import INCEPTION_V2_PATH
from PIL import Image
from datasets.dataset_list import get_dataset_splits
from common.metrics import get_prototypes, top1_gzsl

Using TensorFlow backend.


In [16]:
model = None
model = InceptionV2Loader()

INFO:tensorflow:Restoring parameters from /mnt/datasets/public/research/cvpr2016_cub/inception_v2.ckpt


INFO:tensorflow:Restoring parameters from /mnt/datasets/public/research/cvpr2016_cub/inception_v2.ckpt


In [17]:
sample_images = ['dog.jpg', 'panda.jpg', 'tinca_tinca.jpg']

for image in sample_images:
    im = Image.open(image).resize((299,299))
    im = np.array(im)
    im = im.reshape(-1,299,299,3).astype(np.float32)
    predict_values, logit_values, embedding = model.predict(im)
    print (np.max(predict_values), np.max(logit_values))
    print (np.argmax(predict_values)-1, np.argmax(logit_values)-1)
    
print(embedding.shape)

0.904487 9.62683
251 251
0.880126 9.28199
388 388
0.740908 8.68079
0 0
(1, 1024)


In [18]:
model = None
model = InceptionV3Loader()

INFO:tensorflow:Restoring parameters from /mnt/datasets/public/research/cvpr2016_cub/inception_v3.ckpt


INFO:tensorflow:Restoring parameters from /mnt/datasets/public/research/cvpr2016_cub/inception_v3.ckpt


In [19]:
sample_images = ['dog.jpg', 'panda.jpg', 'tinca_tinca.jpg']

for image in sample_images:
    im = Image.open(image).resize((299,299))
    im = np.array(im)
    im = im.reshape(-1,299,299,3).astype(np.float32)
    predict_values, logit_values, embedding = model.predict(im)
    print (np.max(predict_values), np.max(logit_values))
    print (np.argmax(predict_values)-1, np.argmax(logit_values)-1)
    
print(embedding.shape)

0.996392 13.0912
251 251
0.950092 10.0441
388 388
0.981881 11.0987
0 0
(1, 2048)


## Test nearest neighbour model loader

In [47]:

model_dir = 'batch_size-32-steps-100002-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101'
log_dir = f"logs/{model_dir}"


model_dir = 'mi_weight=0.4;consistency_loss=CLASSIFIER;embedding_size=512;num_texts=50;num_images=5;train_scheme=PAIRWISE;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=0;train_batch_size=32;modality_interaction=None'
# model_dir = 'mi_weight=0.4;consistency_loss=CLASSIFIER;embedding_size=512;num_texts=10;num_images=5;train_scheme=FEWSHOT;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=0;train_batch_size=32;modality_interaction=None'
log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190419_115640_consistency_loss_embedding_size_num_images_train_scheme_text_feature_extractor_number_of_steps_repeat_train_batch_size_modality_interaction_mi_weight_num_texts_seen_unseen_test/{model_dir}"

# model_dir = 'mi_weight=0.4;embedding_size=512;dataset=xian2018_flowers;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=0;modality_interaction=NONE'
# log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190427_122014_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_xian2018_flowers_test/{model_dir}"


model_dir = 'mi_weight=0.5;embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE'
log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190506_124706_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_test_split_train_split_GZSL_results_test_split/{model_dir}"

model_dir = 'embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE'
log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/{model_dir}"



metric_model = MetricLoader(model_path=log_dir, batch_size_image=100, batch_size_text=200)

feature_model = ModelLoader(model_path=log_dir, batch_size=None,
                            num_images=metric_model.flags.num_images, num_texts=metric_model.flags.num_texts, 
                            max_text_len=30)


INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.


Loading model
Loading model
INFO:tensorflow:Restoring parameters from /mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/train/model-100000


INFO:tensorflow:Restoring parameters from /mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/train/model-100000


In [48]:
dataset_splits = get_dataset_splits(dataset_name=metric_model.flags.dataset, data_dir=metric_model.flags.data_dir,
                                    splits=[metric_model.flags.train_split, 
                                            metric_model.flags.test_split+"_seen", 
                                            metric_model.flags.test_split+"_unseen"], 
                                    flags=metric_model.flags)

INFO:root:Loading split trainval
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_trainval_xian2017.pkl
INFO:root:Loaded cache in 6.063078 sec
INFO:root:Loading split test_seen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_test_seen_xian2017.pkl
INFO:root:Loaded cache in 2.655828 sec
INFO:root:Loading split test_unseen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_test_unseen_xian2017.pkl
INFO:root:Loaded cache in 3.430594 sec


In [49]:
results_eval, embeddings = feature_model.eval_acc_gzsh(train_loader=dataset_splits[metric_model.flags.train_split], 
                                                       test_loader_seen=dataset_splits[metric_model.flags.test_split+"_seen"],
                                                       test_loader_unseen=dataset_splits[metric_model.flags.test_split+"_unseen"],
                                                       batch_size=32, seen_adjustment=0.02) # 0.015
results_eval



INFO:root:Computing train embeddings
221it [00:06, 35.94it/s]
INFO:root:Computing test embeddings, unseen
93it [00:02, 39.07it/s]
INFO:root:Computing test embeddings, seen
56it [00:01, 37.95it/s]
INFO:root:Computing generalized zero-shot performance metrics
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.
 17%|█▋        | 1/6 [00:00<00:00,  6.91it/s]

Loading model
****************
UNSEEN images test
****************
STD seen prototype:
62.7574590858
STD unseen prototype:
63.0824614458


 50%|█████     | 3/6 [00:00<00:00,  7.06it/s]

STD seen prototype:
61.7169943359
STD unseen prototype:
61.6395415988
STD seen prototype:
61.64822423
STD unseen prototype:
61.6026712354


 83%|████████▎ | 5/6 [00:00<00:00,  7.13it/s]

...

100%|██████████| 6/6 [00:00<00:00,  9.29it/s]
INFO:root:Computing classical zero-shot performance metrics, test
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.


STD seen prototype:
60.3658705275
STD unseen prototype:
63.3989551743
Loading model


INFO:root:Computing classical zero-shot performance metrics, train
INFO:root:Searching for '/mnt/scratch/boris/experiments_zeroshot_pairwise/190508_220717_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_test_split_train_split_GZSL_results_test_split/embedding_size=512;test_split=test;train_split=trainval;dataset=xian2017_cub;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=9;modality_interaction=NONE/params.json'
INFO:root:Loading existing params.


Loading model


{'train_AP@50/#sentences10': 0.32813333333333333,
 'train_Top-1 Acc/#sentences10': 0.46605845184163047,
 'train_AP@50/#sentences50': 0.44586666666666663,
 'train_Top-1 Acc/#sentences50': 0.70231396771063315,
 'train_AP@50/#sentences100': 0.46293333333333331,
 'train_Top-1 Acc/#sentences100': 0.74451485044499388,
 'train_AP@50/#sentences200': 0.46879999999999994,
 'train_Top-1 Acc/#sentences200': 0.77205143990811653,
 'train_AP@50/#sentences400': 0.47426666666666678,
 'train_Top-1 Acc/#sentences400': 0.78884844868922821,
 'train_AP@50/#sentences1000': 0.47373333333333334,
 'train_Top-1 Acc/#sentences1000': 0.7934936279164273,
 'test_AP@50/#sentences10': 0.33839999999999998,
 'test_Top-1 Acc/#sentences10': 0.42058009334452878,
 'test_AP@50/#sentences50': 0.41639999999999994,
 'test_Top-1 Acc/#sentences50': 0.58912191280667392,
 'test_AP@50/#sentences100': 0.4204,
 'test_Top-1 Acc/#sentences100': 0.63651968174850848,
 'test_AP@50/#sentences200': 0.43480000000000002,
 'test_Top-1 Acc/#sent

## Compute correction factor on the validation subset, CUB

In [140]:

model_dir = 'batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101'
log_dir = f"logs/{model_dir}"

# model_dir = 'mi_weight=0.4;embedding_size=512;dataset=xian2018_flowers;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=0;modality_interaction=FILM_T'
# log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190427_122014_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_xian2018_flowers_test/{model_dir}"


metric_model = MetricLoader(model_path=log_dir, batch_size_image=100, batch_size_text=200)

feature_model = ModelLoader(model_path=log_dir, batch_size=None,
                            num_images=metric_model.flags.num_images, num_texts=metric_model.flags.num_texts, 
                            max_text_len=30)


INFO:root:Searching for 'logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.
INFO:root:Searching for 'logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.


Loading model
Loading model
INFO:tensorflow:Restoring parameters from logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/train/model-100000


INFO:tensorflow:Restoring parameters from logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/train/model-100000


In [141]:
dataset_splits = get_dataset_splits(dataset_name=metric_model.flags.dataset, data_dir=metric_model.flags.data_dir,
                                    splits=[metric_model.flags.train_split, 
                                            metric_model.flags.test_split+"_seen", 
                                            metric_model.flags.test_split+"_unseen"], 
                                    flags=metric_model.flags)

INFO:root:Loading split train
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_train_xian2017.pkl
INFO:root:Loaded cache in 4.652322 sec
INFO:root:Loading split val_seen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_val_seen_xian2017.pkl
INFO:root:Loaded cache in 2.488875 sec
INFO:root:Loading split val_unseen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_val_unseen_xian2017.pkl
INFO:root:Loaded cache in 3.572294 sec


In [149]:
results_eval, embeddings = feature_model.eval_acc_gzsh(train_loader=dataset_splits[metric_model.flags.train_split], 
                                                       test_loader_seen=dataset_splits[metric_model.flags.test_split+"_seen"],
                                                       test_loader_unseen=dataset_splits[metric_model.flags.test_split+"_unseen"],
                                                       batch_size=32, seen_adjustment=0.02) # 0.015
results_eval


INFO:root:Computing train embeddings
147it [00:03, 39.37it/s]
INFO:root:Computing test embeddings, unseen
93it [00:02, 42.48it/s]
INFO:root:Computing test embeddings, seen
37it [00:00, 40.24it/s]
INFO:root:Computing generalized zero-shot performance metrics
INFO:root:Searching for 'logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.
 17%|█▋        | 1/6 [00:00<00:00,  8.70it/s]

Loading model
****************
UNSEEN images test
****************
STD seen prototype:
45.8416163628
STD unseen prototype:
45.7272962191


 50%|█████     | 3/6 [00:00<00:00,  9.06it/s]

STD seen prototype:
45.2667278979
STD unseen prototype:
45.3625948194
STD seen prototype:
45.1561514952
STD unseen prototype:
45.218320328


 83%|████████▎ | 5/6 [00:00<00:00,  9.37it/s]

...

INFO:root:Loading existing params.


Loading model


INFO:root:Computing classical zero-shot performance metrics, train
INFO:root:Searching for 'logs/batch_size-32-steps-100006-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.


Loading model


{'train_AP@50/#sentences10': 0.38319999999999993,
 'train_Top-1 Acc/#sentences10': 0.53894202808253033,
 'train_AP@50/#sentences50': 0.46140000000000003,
 'train_Top-1 Acc/#sentences50': 0.78636570956544216,
 'train_AP@50/#sentences100': 0.49800000000000005,
 'train_Top-1 Acc/#sentences100': 0.83721254914492649,
 'train_AP@50/#sentences200': 0.49800000000000005,
 'train_Top-1 Acc/#sentences200': 0.86598500401185918,
 'train_AP@50/#sentences400': 0.502,
 'train_Top-1 Acc/#sentences400': 0.88435971215905607,
 'train_AP@50/#sentences1000': 0.50140000000000007,
 'train_Top-1 Acc/#sentences1000': 0.88538158213947671,
 'test_AP@50/#sentences10': 0.27399999999999997,
 'test_Top-1 Acc/#sentences10': 0.4156715532706311,
 'test_AP@50/#sentences50': 0.28000000000000003,
 'test_Top-1 Acc/#sentences50': 0.56310700548211112,
 'test_AP@50/#sentences100': 0.29239999999999999,
 'test_Top-1 Acc/#sentences100': 0.61088339272974501,
 'test_AP@50/#sentences200': 0.28999999999999998,
 'test_Top-1 Acc/#sente

In [139]:
89.31 / 83.13

1.0743413929989174

In [61]:
embeddings[0]

array([[ 2.37279391,  0.02427058, -2.69599867, ...,  0.0203332 ,
        -1.44127524,  0.30021363],
       [ 1.02579761, -1.42297983, -0.12612197, ..., -0.93158758,
         0.07975166, -2.19573784],
       [ 1.19220066,  1.97308278, -2.59228873, ..., -0.53009856,
         0.70677149, -1.83169401],
       ..., 
       [-0.99557626,  0.10689936,  0.21294507, ...,  0.28846174,
        -0.97834027, -1.66222882],
       [-0.57412851,  2.34132791, -1.39146113, ...,  1.85419559,
        -2.26026034, -2.53814721],
       [-0.65821981,  2.30379725,  0.2592791 , ...,  1.03106701,
         0.76612294, -1.74685097]], dtype=float32)

## Compute correction factor on the validation subset, FLOWERS

In [82]:

model_dir = 'batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101'
log_dir = f"logs/{model_dir}"

# model_dir = 'mi_weight=0.4;embedding_size=512;dataset=xian2018_flowers;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=0;modality_interaction=FILM_T'
# log_dir = f"/mnt/scratch/boris/experiments_zeroshot_pairwise/190427_122014_embedding_size_dataset_text_feature_extractor_number_of_steps_repeat_modality_interaction_mi_weight_xian2018_flowers_test/{model_dir}"


metric_model = MetricLoader(model_path=log_dir, batch_size_image=100, batch_size_text=200)

feature_model = ModelLoader(model_path=log_dir, batch_size=None,
                            num_images=metric_model.flags.num_images, num_texts=metric_model.flags.num_texts, 
                            max_text_len=30)


INFO:root:Searching for 'logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.
INFO:root:Searching for 'logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.


Loading model
Loading model
INFO:tensorflow:Restoring parameters from logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/train/model-100000


INFO:tensorflow:Restoring parameters from logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/train/model-100000


In [83]:
dataset_splits = get_dataset_splits(dataset_name=metric_model.flags.dataset, data_dir=metric_model.flags.data_dir,
                                    splits=[metric_model.flags.train_split, 
                                            metric_model.flags.test_split+"_seen", 
                                            metric_model.flags.test_split+"_unseen"], 
                                    flags=metric_model.flags)

INFO:root:Loading split train
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_flowers/split_train_xian2018.pkl
INFO:root:Loaded cache in 3.845049 sec
INFO:root:Loading split val_seen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_flowers/split_val_seen_xian2018.pkl
INFO:root:Loaded cache in 1.757812 sec
INFO:root:Loading split val_unseen
INFO:root:Loading cached file /mnt/datasets/public/research/cvpr2016_flowers/split_val_unseen_xian2018.pkl
INFO:root:Loaded cache in 1.757477 sec


In [117]:
results_eval, embeddings = feature_model.eval_acc_gzsh(train_loader=dataset_splits[metric_model.flags.train_split], 
                                                       test_loader_seen=dataset_splits[metric_model.flags.test_split+"_seen"],
                                                       test_loader_unseen=dataset_splits[metric_model.flags.test_split+"_unseen"],
                                                       batch_size=32, seen_adjustment=0.03) # 0.015
results_eval


INFO:root:Computing train embeddings
147it [00:03, 39.66it/s]
INFO:root:Computing test embeddings, unseen
37it [00:00, 40.03it/s]
INFO:root:Computing test embeddings, seen
37it [00:00, 40.72it/s]
INFO:root:Computing generalized zero-shot performance metrics
INFO:root:Searching for 'logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.
 33%|███▎      | 2/6 [00:00<00:00, 18.36it/s]

Loading model
STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
97.6802033168
STD unseen prototype, unseen image:
93.8505447693
STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
94.5851890172
STD unseen prototype, unseen image:
89.3163926371
STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
94.1561617375
STD unseen prototype, unseen image:
88.8894088512


  0%|          | 0/6 [00:00<?, ?it/s]

STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
93.7038158995
STD unseen prototype, unseen image:
88.9308668632
STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
93.668974371
STD unseen prototype, unseen image:
88.6498308789
STD seen prototype, seen image:
nan
STD unseen prototype, seen image:
nan
STD seen prototype, unseen image:
93.6536381085
STD unseen prototype, unseen image:
88.6574133718
STD seen prototype, seen image:
98.9347098598
STD unseen prototype, seen image:
95.8301039984
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan


 50%|█████     | 3/6 [00:00<00:00, 20.61it/s]

STD seen prototype, seen image:
94.7817565133
STD unseen prototype, seen image:
92.3510630444
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan
STD seen prototype, seen image:
94.3990874737
STD unseen prototype, seen image:
91.5304065615
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan
STD seen prototype, seen image:
94.0668049553
STD unseen prototype, seen image:
91.5184376361
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan
STD seen prototype, seen image:
93.9170031821
STD unseen prototype, seen image:
91.4339155959
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan
STD seen prototype, seen image:
93.9533600582
STD unseen prototype, seen image:
91.3455755398
STD seen prototype, unseen image:
nan
STD unseen prototype, unseen image:
nan


100%|██████████| 6/6 [00:00<00:00, 20.43it/s]
INFO:root:Computing classical zero-shot performance metrics, test
INFO:root:Searching for 'logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.
INFO:root:Computing classical zero-shot performance metrics, train
INFO:root:Searching for 'logs/batch_size-32-steps-100005-lr-0.1005-opt-sgd-weight_decay-0.001-nfilt-64-image_feature_extractor-resnet101/params.json'
INFO:root:Loading existing params.


Loading model
Loading model


{'train_AP@50/#sentences10': 0.68225806451612903,
 'train_Top-1 Acc/#sentences10': 0.67822533972704924,
 'train_AP@50/#sentences50': 0.83935483870967753,
 'train_Top-1 Acc/#sentences50': 0.88230237241316212,
 'train_AP@50/#sentences100': 0.85193548387096774,
 'train_Top-1 Acc/#sentences100': 0.90395697481436288,
 'train_AP@50/#sentences200': 0.87161290322580653,
 'train_Top-1 Acc/#sentences200': 0.92529027982008238,
 'train_AP@50/#sentences400': 0.87387096774193562,
 'train_Top-1 Acc/#sentences400': 0.92324004314025354,
 'train_AP@50/#sentences1000': 0.87516129032258072,
 'train_Top-1 Acc/#sentences1000': 0.92492238795771076,
 'test_AP@50/#sentences10': 0.28800000000000003,
 'test_Top-1 Acc/#sentences10': 0.38084491775671431,
 'test_AP@50/#sentences50': 0.32000000000000001,
 'test_Top-1 Acc/#sentences50': 0.51591351718579948,
 'test_AP@50/#sentences100': 0.35099999999999998,
 'test_Top-1 Acc/#sentences100': 0.57218566322697473,
 'test_AP@50/#sentences200': 0.36100000000000004,
 'test_T

In [14]:
seen_unseen_text_embeddings, image_embeddings_train, image_embeddings_test = embeddings

In [16]:
seen_unseen_subsets = {}
seen_unseen_subsets['seen'] = list(set(dataset_splits[metric_model.flags.train_split].image_classes))
seen_unseen_subsets['unseen'] = list(set(dataset_splits[metric_model.flags.test_split].image_classes))

seen_unseen_labels = np.concatenate(
    [dataset_splits[metric_model.flags.train_split].image_classes, dataset_splits[metric_model.flags.test_split].image_classes], axis=0)

class_ids_query = dataset_splits[metric_model.flags.test_split].image_classes


In [17]:
current_num_texts = 1000

prototypes = get_prototypes(embeddings=seen_unseen_text_embeddings, labels=seen_unseen_labels, 
                            vectors_per_protype=current_num_texts)

class_ids_prototypes = np.array(list(prototypes.keys()))
array_prototypes = np.array(list(prototypes.values()))

seen_unseen_flag = np.isin(class_ids_prototypes, seen_unseen_subsets['seen']).astype(np.float32)

In [18]:
dist = metric_model.predict_all(image_embeddings=image_embeddings_test, text_embeddings=array_prototypes)
dist = dist.transpose()
dist = dist * dist
dist = dist * (0.0*seen_unseen_flag[:,None] + 1.0)

In [19]:
metrics = {}
nn_idxs = np.argmin(dist, axis=0)
top1_acc_matches = {}
top1_acc_counts = {}
top1_acc = {} 
for query_id, nn_idx in enumerate(nn_idxs):
    actual_class = class_ids_query[query_id]
    nearest_class = class_ids_prototypes[nn_idx]
    top1_acc_matches[actual_class] = top1_acc_matches.get(actual_class, 0.0) + np.float(nearest_class == actual_class)
    top1_acc_counts[actual_class] = top1_acc_counts.get(actual_class, 0.0) + 1

for key in top1_acc_matches.keys():
    top1_acc[key] = top1_acc_matches[key] / top1_acc_counts[key]

metrics['Top-1 Acc/#sentences%d'%(10*current_num_texts)] = np.array(list(top1_acc.values())).mean()

metrics

{'Top-1 Acc/#sentences10000': 0.40651929037993684}

In [22]:
top1_gzsl(support_embeddings=seen_unseen_text_embeddings, 
          query_embeddings=image_embeddings_test, 
          class_ids_support=seen_unseen_labels, 
          class_ids_query=dataset_splits[metric_model.flags.test_split].image_classes, 
          num_texts=[1, 3, 10, 20, 30, 40, 50, 100], 
          seen_unseen_subsets=seen_unseen_subsets, distance_metric=metric_model)


100%|██████████| 8/8 [00:01<00:00,  7.32it/s]


{'Top-1 Acc/#sentences10': 0.2819812909754178,
 'Top-1 Acc/#sentences30': 0.42396133527416019,
 'Top-1 Acc/#sentences100': 0.50692222609057291,
 'Top-1 Acc/#sentences200': 0.52879963346939074,
 'Top-1 Acc/#sentences300': 0.54425090994636993,
 'Top-1 Acc/#sentences400': 0.55583511289720389,
 'Top-1 Acc/#sentences500': 0.55823191321101295,
 'Top-1 Acc/#sentences1000': 0.56060498938479053}

In [23]:
top1_gzsl(support_embeddings=seen_unseen_text_embeddings, 
          query_embeddings=image_embeddings_train, 
          class_ids_support=seen_unseen_labels, 
          class_ids_query=dataset_splits[metric_model.flags.train_split].image_classes, 
          num_texts=[1, 3, 10, 20, 30, 40, 50, 100], 
          seen_unseen_subsets=seen_unseen_subsets, distance_metric=metric_model)


100%|██████████| 8/8 [00:02<00:00,  3.40it/s]


{'Top-1 Acc/#sentences10': 0.35301990756053186,
 'Top-1 Acc/#sentences30': 0.45103795607907587,
 'Top-1 Acc/#sentences100': 0.50666833382992293,
 'Top-1 Acc/#sentences200': 0.54526526941628528,
 'Top-1 Acc/#sentences300': 0.53615094210743031,
 'Top-1 Acc/#sentences400': 0.54992179908153682,
 'Top-1 Acc/#sentences500': 0.54932806065160544,
 'Top-1 Acc/#sentences1000': 0.5510017521452415}

In [315]:
unseen_text_embeddings = seen_unseen_text_embeddings[len(dataset_splits[nn_model.flags.train_split].image_classes):]
unseen_labels = seen_unseen_labels[len(dataset_splits[nn_model.flags.train_split].image_classes):]

In [316]:
nn_model = NnModelLoader(model_path=log_dir_nn, batch_size_image=10, batch_size_text=len(unseen_text_embeddings))

INFO:root:Searching for '/mnt/home/boris/experiments_zeroshot_pairwise/190409_000054_embedding_size_film_weight_decay_postmult_text_feature_extractor_number_of_steps_repeat_modality_interaction_num_texts_dropout_encoder_decoder_test/embedding_size=512;film_weight_decay_postmult=0.1;num_texts=10;dropout=0.25;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=1;modality_interaction=FILM/params.json'
INFO:root:Loading existing params.


Loading model
INFO:tensorflow:Restoring parameters from /mnt/home/boris/experiments_zeroshot_pairwise/190409_000054_embedding_size_film_weight_decay_postmult_text_feature_extractor_number_of_steps_repeat_modality_interaction_num_texts_dropout_encoder_decoder_test/embedding_size=512;film_weight_decay_postmult=0.1;num_texts=10;dropout=0.25;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=1;modality_interaction=FILM/train/model-100000


INFO:tensorflow:Restoring parameters from /mnt/home/boris/experiments_zeroshot_pairwise/190409_000054_embedding_size_film_weight_decay_postmult_text_feature_extractor_number_of_steps_repeat_modality_interaction_num_texts_dropout_encoder_decoder_test/embedding_size=512;film_weight_decay_postmult=0.1;num_texts=10;dropout=0.25;text_feature_extractor=cnn_bi_lstm;number_of_steps=100001;repeat=1;modality_interaction=FILM/train/model-100000


In [317]:
dist = nn_model.predict_all(image_embeddings=image_embeddings_test, text_embeddings=unseen_text_embeddings)
dist = dist.transpose()
dist = dist * dist
# dist = dist * (0.02*seen_unseen_flag[:,None] + 1.0)

In [318]:
metrics = {}
nn_idxs = np.argmin(dist, axis=0)
top1_acc_matches = {}
top1_acc_counts = {}
top1_acc = {} 
for query_id, nn_idx in enumerate(nn_idxs):
    actual_class = class_ids_query[query_id]
    nearest_class = unseen_labels[nn_idx]
    top1_acc_matches[actual_class] = top1_acc_matches.get(actual_class, 0.0) + np.float(nearest_class == actual_class)
    top1_acc_counts[actual_class] = top1_acc_counts.get(actual_class, 0.0) + 1

for key in top1_acc_matches.keys():
    top1_acc[key] = top1_acc_matches[key] / top1_acc_counts[key]

metrics['Top-1 Acc/#sentences%d'%(10*current_num_texts)] = np.array(list(top1_acc.values())).mean()

metrics

{'Top-1 Acc/#sentences10000': 0.60447010585803074}

In [221]:
seen_unseen_text_embeddings.shape

(11788, 512)

In [18]:
logits.shape

(100, 150)

In [7]:
# import gensim.downloader as api

# start = time.time()
# text_model = model = api.load("word2vec-google-news-300")
# print("time loading weights:", time.time()-start)

In [8]:
# import gensim

# start = time.time()
# text_model1 = gensim.models.KeyedVectors.load_word2vec_format(
#     "/mnt/datasets/public/research/GoogleNews_vectors_negative300/GoogleNews-vectors-negative300.bin.gz", 
#     binary=True)
# print("time loading weights:", time.time()-start)

In [9]:
# text_model1['computer', 'IBM', 'bird', 'belly'].shape
# text_model1.wv['computer']
# text_model1.vocab


In [10]:
log_dir = "logs/batch_size-24-steps-100000-lr-0.001-opt-adam-weight_decay-0.0005-nfilt-64-image_feature_extractor-simple_res_net"
log_dir = "logs/batch_size-24-steps-100000-lr-0.001-opt-adam-weight_decay-0.0005-nfilt-64-image_feature_extractor-inception_v3"

exp_dir = "/mnt/home/boris/experiments_zeroshot_pairwise/"
log_dir = exp_dir + \
"190206_164521_repeat_optimizer_num_images_number_of_steps_train_batch_size_num_texts_metric_multiplier_init_word2vec_test/repeat=0;optimizer=sgd;num_images=3;number_of_steps=50000;train_batch_size=64;num_texts=3;metric_multiplier_init=5/"


In [11]:
# !ls "/mnt/home/boris/experiments_zeroshot_pairwise/190201_145439_repeat_optimizer_num_images_lr_decay_rate_number_of_steps_init_learning_rate_train_batch_size_word_embed_dim_num_texts_sgd_optimizer/" + \
# "190201_145439_repeat_optimizer_num_images_lr_decay_rate_number_of_steps_init_learning_rate_train_batch_size_word_embed_dim_num_texts_sgd_optimizer/repeat=0;optimizer=sgd;num_images=3;word_embed_dim=256;number_of_steps=50000;init_learning_rate=0.1;train_batch_size=24;lr_decay_rate=10;num_texts=5"

os.listdir(log_dir + '/train')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/home/boris/experiments_zeroshot_pairwise/190206_164521_repeat_optimizer_num_images_number_of_steps_train_batch_size_num_texts_metric_multiplier_init_word2vec_test/repeat=0;optimizer=sgd;num_images=3;number_of_steps=50000;train_batch_size=64;num_texts=3;metric_multiplier_init=5//train'

In [ ]:
flags = Namespace(load_and_save_params({}, log_dir))

In [ ]:
train_loader = Cvpr2016CubLoader(split='trainval')
train_loader.load_cached()

In [ ]:
train_loader.image_embeddings.shape

In [6]:
test_loader = Cvpr2016CubLoader(split='test')
test_loader.load_cached()


Loading split test
Loading cached file /mnt/datasets/public/research/cvpr2016_cub/split_test.pkl
Loaded cache in 14.548067331314087 sec


In [8]:
test_loader.image_embeddings.shape

(2933, 10, 1024)

In [ ]:
model = ModelLoader(model_path=log_dir, 
                    batch_size=None, num_images=10, num_texts=10, max_text_len=30)

In [ ]:
# image_embeddings, text_embeddings = model.eval_acc(data_set=test_loader, batch_size=5)

ap50, image_embeddings, text_embeddings = model.eval_acc(data_set=test_loader, batch_size=10)
print(ap50)

In [ ]:
from common.metrics import get_prototypes, ap_at_k_prototypes

metrics = ap_at_k_prototypes(support_embeddings=text_embeddings, query_embeddings=image_embeddings, 
                   class_ids=test_loader.image_classes, k=50, num_texts=[1, 2, 5, 10, 20, 30, 40])



metrics

In [ ]:
model = ModelLoader(model_path=log_dir, 
                    batch_size=32, num_images=flags.num_images, num_texts=flags.num_texts, 
                    max_text_len=train_loader.max_text_len)
model.eval_acc_batch(data_set=test_loader)

In [ ]:
ap_at50, top1_acc = ap_at_k(support_embeddings=text_embeddings, query_embeddings=image_embeddings, 
                 class_ids=test_loader.image_classes)
print(ap_at50, top1_acc)

In [ ]:
kdtree = cKDTree(text_embeddings)
ap50 = 0.0
print("Computing AP@50")
for query_id, query_embedding in enumerate(tqdm(image_embeddings)):
    nns, nn_idxs = kdtree.query(query_embedding, k=50)
    nn_idxs = nn_idxs.tolist()
    
    nearest_classes = np.array(test_loader.image_classes)[nn_idxs]
    ap50 += sum([c == test_loader.image_classes[query_id] for c in nearest_classes])/50
ap50 /= len(image_embeddings)

ap50

# query_id = 0
# nns, nn_idxs = kdtree.query(image_embeddings[query_id], k=50)
#     nns_total[query_id] = nns
#     nn_idxs_total.append(nn_idxs)

# nn_idxs

# print(np.array(test_loader.image_classes)[nn_idxs])

# print(test_loader.image_classes[query_id])

# test_loader.image_classes[query_id]

# len(test_loader.image_classes)
# len(image_embeddings)
# test_loader.image_classes

In [ ]:
# feed_dict = {model.images_pl: images.astype(dtype=np.float32),
#                      model.text_pl: texts,
#                      model.text_len_pl: text_lengths}

# image_embeddings, text_embeddings = model.sess.run([model.image_embeddings, model.text_embeddings], feed_dict)
# print(text_embeddings.sum(axis=1))

In [ ]:

for images, texts, text_lengths in train_loader.sequential_evaluation_batches(batch_size=5, num_images=10, num_texts=10):
    a = model.predict(images, texts, text_lengths)
    a[0] == 0
    break
    
print(a[1]==0)

In [ ]:
images, texts, text_lengths, _ = train_loader.next_batch(batch_size=2, num_images=10, num_texts=10)
model.predict(images, texts, text_lengths)

In [ ]:
a = {'a': 0, 'b':1}

In [ ]:
list(a.values())[0]